In [3]:
import os
import numpy as np
from datetime import datetime
import glob
from rtree import index
import math
import pandas as pd

In [78]:
path = '/Users/sabine.a.joseph/Downloads/'
df = pd.read_csv(path + 'gdelt_20140101_20140131.csv')

In [119]:
# get and format gridcell data
df_grid = pd.read_csv('/Users/sabine.a.joseph/Desktop/Gridcells_with_countryinfo.csv', sep = ';')

def correct_coordinate_format(df, colname_list):
    for i in range(0, len(colname_list)):
        df[colname_list[i]] = [(float(df[colname_list[i]][j][:5])) for j in range (0, len(df[colname_list[i]]))]
    return df

df_grid = correct_coordinate_format(df_grid, ['xmin', 'xmax', 'ymin', 'ymax'])

In [121]:
def rtree_index_to_bbox_column(df_lon_col, df_lat_col):    
    idx = index.Index()
    # create rtree index, contains all bounding boxes
    for i in range(0, len(df_grid.id)):
        # if interleaved is True: xmin, ymin, xmax, ymax
        idx.insert(i, (df_grid.xmin[i], df_grid.ymin[i], df_grid.xmax[i], df_grid.ymax[i]))
    
    # retrieve intersection idx for each coordinate pair
    return [(list(idx.intersection((float(df_lon_col[i]), float(df_lat_col[i]), 
                                    float(df_lon_col[i]), float(df_lat_col[i])))))[0]
            if math.isnan(df_lat_col[i]) is False and (list(idx.intersection((float(df_lon_col[i]), float(df_lat_col[i]), 
                                                                          float(df_lon_col[i]), float(df_lat_col[i])))))
            else np.nan for i in range (0, df.shape[0])]

df['bbox'] = rtree_index_to_bbox_column(df.Actor1Geo_Long, df.Actor1Geo_Lat)

In [124]:
# url and event ID duplicate removal
# create new columns for protest, material conflict, rebellion, radicalism
# cast Goldstein to float
def EoI_columns(df, col_name_dict):
    # max eventid for each url 
    if col_name_dict['url_name'] and col_name_dict['eventID_name'] is not None: 
        gdelt_max_id = df.groupby(col_name_dict['url_name'])[col_name_dict['eventID_name']].max()
        # keep only max ids to remove duplicates
        df = df[df[col_name_dict['eventID_name']].isin(gdelt_max_id)]
    if col_name_dict['root_code_name'] is not None: 
        df['protest'] = np.where(df[col_name_dict['root_code_name']]==14, 1, 0)
    if col_name_dict['quad_class_name'] is not None:
        df['material_conflict'] = np.where(df[col_name_dict['quad_class_name']]==int(4), 1, 0)   
    if col_name_dict['actor_name'] is not None: 
        df['rebellion'] = np.where(df[col_name_dict['actor_name']].isin(['REB','SEP','INS']), 1, 0)
    if col_name_dict['Actor1Code'] and col_name_dict['Actor2Code'] and col_name_dict['Actor3Code'] is not None: 
        df['radicalism'] = np.where(np.logical_or.reduce((df[col_name_dict['Actor1Code']]=='RAD',
                                                          df[col_name_dict['Actor2Code']]=='RAD',
                                                          df[col_name_dict['Actor3Code']]=='RAD')),1, 0)
    if 'goldstein_name' in col_name_dict:
        df['GoldsteinScale'] = df[col_name_dict['goldstein_name']].apply(lambda x : float(x))
    return df

# GDELT column names
col_names = {
    'eventID_name' : 'GLOBALEVENTID',
    'root_code_name' : 'EventRootCode',
    'quad_class_name': 'QuadClass',
    'geo_country_name' : 'SourceActorFull',
    'geo_region_name' : 'ActionGeo_CountryCode',
    'actor_name' : 'Actor1Type1Code',
    'url_name' : 'SOURCEURL',
    'goldstein_name' : 'GoldsteinScale',
    'date_name' : 'SQLDATE',
    'Actor1Code': 'Actor1Type1Code',
    'Actor2Code': 'Actor1Type2Code',
    'Actor3Code': 'Actor1Type3Code'
}

df.QuadClass = [int(df.QuadClass[i]) for i in range(0, df.shape[0])]
df.EventRootCode = [int(df.EventRootCode[i]) for i in range(0, df.shape[0])]

df = EoI_columns(df, col_names)
#df = df.reset_index()

In [136]:
#df.columns = df.columns.get_level_values(0)

df.head()

,level_0,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,...,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,bbox,rebellion,radicalism,protest,material_conflict,count_num_daily_events
SQLDATE,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0,7,281451791,2013-01-01,201301,2013,2013.0027,BHR,MANAMA,BHR,...,50.5831,-784833,20140101,http://www.gulf-daily-news.com/NewsDetails.asp...,NaN,0,0,0,0,1
2013-01-01,1,8,281451858,2013-01-01,201301,2013,2013.0027,CVL,TOURIST,NaN,...,57.0000,MU,20140101,http://www.omantribune.com/index.php?page=news...,NaN,0,0,0,0,1
2013-01-01,2,20,281452101,2013-01-01,201301,2013,2013.0027,KWT,KUWAIT,KWT,...,47.6581,KU,20140101,http://www.telegram.com/article/20140101/NEWS/...,NaN,0,0,0,1,1
2013-01-01,3,26,281452279,2013-01-01,201301,2013,2013.0027,SYR,SYRIA,SYR,...,38.0000,SY,20140101,http://timesofindia.indiatimes.com/tech/tech-n...,NaN,0,0,0,0,1
2013-01-01,4,29,281452288,2013-01-01,201301,2013,2013.0027,TUR,TURKEY,TUR,...,35.6592,-749675,20140101,http://www.hurriyetdailynews.com/turkish-pm-er...,NaN,0,0,0,0,1


In [132]:
# save raw df
def df_to_csv(df, path, filename):
    df.to_csv(path + filename)

# example input and call
path = '/Users/sabine.a.joseph/Desktop/'
csv_name = 'GDELT_NaMo_subset_1Mo.csv'
df_to_csv(df, path, csv_name) 

In [135]:
# date column to datetime index
df = df.reset_index(drop=True)
df.SQLDATE = [str(df.SQLDATE[i])[:-12] for i in range (0, len(df.SQLDATE)) if i is not None]
#df.SQLDATE = [str(df.SQLDATE[i])[:-10] for i in range (0, len(df.SQLDATE)) if i is not None]

def str_to_datetime(col_name, dateformat):
    return [datetime.strptime(str(df[col_name][i]), dateformat) for i in range(0, df.shape[0]) if i is not None]

df_datestring_column_name = 'SQLDATE'
dateformat = '%Y-%m'
df[df_datestring_column_name] = str_to_datetime(df_datestring_column_name, dateformat)
df.index = df[df_datestring_column_name]

In [152]:
# aggregate per country / bbox and month
# index needs to be datetime
# enter country_col_name as geo-switch: takes country code or bbox
def agg_by_geo_by_month(df, agg_dict, country_col_name):
    agg_df = df.groupby([df.index, country_col_name]).agg(agg_dict)
    agg_df = agg_df.reset_index()
    agg_df.columns = agg_df.columns.get_level_values(0)
    return agg_df
    
df['count_num_daily_events'] = 1 

# create aggregates
aggregations = {
    'protest' : {'protest_events': 'sum'},
    'material_conflict' : {'material_conflict': 'sum'},
    'rebellion' : {'rebellion_events': 'sum'},
    'GoldsteinScale' : {
    'gs_median': 'median',
    'gs_min': lambda x: min(x),
    'gs_max': lambda x: max(x)},
    'AvgTone' : {
    'at_median': 'median',
    'at_min': lambda x: min(x),
    'at_max': lambda x: max(x)},
    'count_num_daily_events' : {'count_num_daily_events': 'sum'},
    'NumMentions' : {'NumMentions': 'sum'},
    'NumSources' : {'NumSources': 'sum'},
    'NumArticles' : {'NumArticles': 'sum'}
}

# geo-level aggregation switch: country vs grid
country_code = 'ActionGeo_CountryCode'
bbox = 'bbox'

agg_df = agg_by_geo_by_month(df, aggregations, country_code) # or 'bbox' for grid level aggregation
# rename columns
agg_df.columns = ['SQLDATE', country_code, 'material_conflict', 'protest', 'gs_median', 'gs_min', 
                  'gs_max', 'NumSources', 'NumMentions', 'at_median', 'at_min', 'at_max', 
                  'count_num_daily_events', 'NumArticles', 'rebellion']


In [153]:
# save agg df 2x
def df_to_csv(df, path, filename):
    df.to_csv(path + filename)

path = '/Users/sabine.a.joseph/Desktop/'

if bbox in agg_df.columns:
    csv_name = 'GDELT_1Mo_NaMo_agg_subset_BBOX.csv'
else:
    csv_name = 'GDELT_1Mo_NaMo_agg_subset_COUNTRY.csv'

df_to_csv(agg_df, path, csv_name) 

In [ ]:
# TO DO:
# loop through all GDELT monthly dataset
# call most functions above in each iteration

# and combine them at the and
# maybe agg once more in case of duplicates across datasets


####
# combine multiple GDELT dfs
def data_to_df(down_dir, col_names, country_codes, filter_col):
    for i in range(0, len(filenames)):
        if i == 0: #create initial df on first loop iteration
            df = pd.read_table(down_dir + '/' + filenames[i][:-4], delim_whitespace=False, 
                               names=col_names)
        else: #concatenate df on each iteration
            df = pd.concat([df, pd.read_table(down_dir + '/' + filenames[i][:-4], delim_whitespace=False, 
                               names = col_names)]) 
            df = df[df[filter_col].isin(country_codes)] 

    df = df[df[filter_col].isin(country_codes)]  
    df = df.reset_index(drop = True)
    return df